In [1]:
from dataclasses import dataclass

In [2]:
@dataclass
class TimedElement():
    start_time: int
    end_time: int
    text: str

class CTMLine(TimedElement):
    pass

class RDAPI(TimedElement):
    pass

In [13]:
aa = [
    CTMLine(0, 5, "foo"),
    CTMLine(5, 10, "foo"),
    CTMLine(10, 15, "foo"),
    CTMLine(21, 25, "foo"),
    CTMLine(30, 35, "foo"),
    CTMLine(41, 45, "foo"),
    CTMLine(50, 55, "foo"),
    CTMLine(50, 65, "foo"),
]

In [4]:
ab = [
    RDAPI(20, 35, "blah"),
    RDAPI(40, 55, "blah"),
]

In [5]:
from copy import deepcopy

In [14]:
i = j = 0
pairs = []
cur = []

while i < len(aa) - 1:
    while j < len(ab) - 1:
        if aa[i].end_time < ab[j].start_time:
            cur.append(aa[i])
        else:
            pairs.append(cur)
            cur = []
            j += 1
            cur.append(aa[i])
        print(aa[i], ab[j], i, j)
        i += 1


CTMLine(start_time=0, end_time=5, text='foo') RDAPI(start_time=20, end_time=35, text='blah') 0 0
CTMLine(start_time=5, end_time=10, text='foo') RDAPI(start_time=20, end_time=35, text='blah') 1 0
CTMLine(start_time=10, end_time=15, text='foo') RDAPI(start_time=20, end_time=35, text='blah') 2 0
CTMLine(start_time=21, end_time=25, text='foo') RDAPI(start_time=40, end_time=55, text='blah') 3 1


KeyboardInterrupt: ignored

In [15]:
pairs

[[CTMLine(start_time=0, end_time=5, text='foo'),
  CTMLine(start_time=5, end_time=10, text='foo'),
  CTMLine(start_time=10, end_time=15, text='foo')]]